In [2]:
import numpy as np
import pickle, random, csv
random.seed(1026847926404610461)

In [3]:
def load(model_file):
    """
    Loads the network from the model_file
    :param model_file: file onto which the network is saved
    :return: the network
    """
    return pickle.load(open(model_file))

Part 1: Implementation of fully connected feed forward neural network

In [13]:
class FullyConnectedFeedForwardNN(object):
    """
    Implementation of a Fully connected feed forward Neural Network. 
    This implementation implements only one hidden layer.
    """
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01, activation = 'relu'):
        """
        Initialize the network with input, output sizes, weights and biases
        :param input_dim: input dim
        :param hidden_size: number of hidden units
        :param output_dim: output dim
        :param learning_rate: learning rate alpha
        :param reg_lambda: regularization rate lambda
        :return: None
        """
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.W_xh = np.random.randn(self.hidden_size, self.input_size) * np.sqrt(2/self.input_size) # Weight matrix for input to hidden
        self.W_hy = np.random.randn(self.output_size, self.hidden_size) * np.sqrt(2/self.hidden_size) # Weight matrix for hidden to output
        self.bias_h = np.zeros((self.hidden_size, 1)) # hidden bias
        self.bias_y = np.zeros((self.output_size, 1)) # output bias
        self.learning_rate = learning_rate
        if(activation == 'sigmoid'):
            self.activation = self._sigmoid
            self.activation_derivative = self._sigmoid_derivative
        elif(activation == 'tanh'):
            self.activation = self._tanh
            self.activation_derivative = self._tanh_derivative
        else:
            self.activation = self._relu
            self.activation_derivative = self._relu_derivative
    
    def _relu(self,Z):
        return np.maximum(Z, 0)
    def _tanh(self,Z):
        return np.tanh(Z)
    def _sigmoid(self,Z):
        return 1/(1+np.exp(-Z))
    
    def _relu_derivative(self,Z):
        Z[Z<=0] = 0
        Z[Z>0] = 1
        return Z
    def _tanh_derivative(self,Z):
        return (1 - Z * Z)
    def _sigmoid_derivative(self,Z):
        return Z*(1-Z)
    
    def _forward_propagation(self, X):
        """
        Performs forward pass of the ANN
        :param X: input
        :return: hidden activations, softmax probabilities for the output
        """
        #hidden_activations = np.tanh(np.dot(self.W_xh, np.reshape(X,(len(X),1))) + self.bias_h)
        Z = np.dot(self.W_xh, np.reshape(X,(len(X),1))) + self.bias_h
        hidden_activations = self.activation(Z)
        y_s = np.exp(np.dot(self.W_hy, hidden_activations) + self.bias_y)
        prob_values = y_s/np.sum(y_s)
        return hidden_activations, prob_values

    def _update_parameter(self, delta_W_xh, delta_bias_h, delta_W_hy, delta_bias_y):
        """
        Update the weights and biases during gradient descent
        :param dWxh: weight derivative from input to hidden
        :param dbh: bias derivative from input to hidden
        :param dWhy: weight derivative from hidden to output
        :param dby: bias derivative from hidden to output
        :return: None
        """
        self.W_xh += -self.learning_rate * delta_W_xh
        self.bias_h += -self.learning_rate * delta_bias_h
        self.W_hy += -self.learning_rate * delta_W_hy
        self.bias_y += -self.learning_rate * delta_bias_y

    def _back_propagation(self, X, target_class, hidden_activations, prob_values):
        """
        Implementation of the backpropagation algorithm
        :param X: input
        :param t: target
        :param h_a: hidden activation from forward pass
        :param probs: softmax probabilities of output from forward pass
        :return: dWxh, dWhy, dbh, dby
        """
        delta_W_xh, delta_W_hy = np.zeros_like(self.W_xh), np.zeros_like(self.W_hy)
        delta_bias_h, delta_bias_y = np.zeros_like(self.bias_h), np.zeros_like(self.bias_y)
        
        delta_y = np.copy(prob_values)
        delta_y[target_class] -= 1
        delta_W_hy = np.dot(delta_y, hidden_activations.T)
        delta_bias_y += delta_y
        
        delta_h = np.dot(self.W_hy.T, delta_y)  # backprop into h
        delta_h_error = self.activation_derivative(hidden_activations) * delta_h # backprop through tanh nonlinearity
        #delta_h_error = (1 - hidden_activations * hidden_activations) * delta_h # backprop through tanh nonlinearity
        delta_bias_h += delta_h_error
        
        delta_W_xh += np.dot(delta_h_error, np.reshape(X, (len(X), 1)).T)
        return delta_W_xh, delta_W_hy, delta_bias_h, delta_bias_y

    def _calc_smooth_loss(self, loss, len_examples):
        """
        Calculate the smoothened loss over the set of examples
        :param loss: loss calculated for a sample
        :param len_examples: total number of samples in training + validation set
        :param regularizer_type: type of regularizer like L1, L2, Dropout
        :return: smooth loss
        """
        return 1./len_examples * loss

    def train(self, inputs, targets, num_epochs,model_file = "NNModel.pkl"):
        """
        Trains the network by performing forward pass followed by backpropagation
        :param inputs: list of training inputs
        :param targets: list of corresponding training targets
        :param validation_data: tuple of (X,y) where X and y are inputs and targets
        :param num_epochs: number of epochs for training the model
        :param regularizer_type: type of regularizer like L1, L2, Dropout
        :return: None
        """
        for k in range(num_epochs):
            loss = 0
            for i in range(len(inputs)):
                # Forward pass
                hidden_activations, prob_values = self._forward_propagation(inputs[i])
                loss += -np.log(prob_values[targets[i], 0])

                # Backpropagation
                delta_W_xh, delta_W_hy, delta_bias_h, delta_bias_y = self._back_propagation(inputs[i], targets[i], hidden_activations, prob_values)

                # Perform the parameter update with gradient descent
                self._update_parameter(delta_W_xh, delta_bias_h, delta_W_hy, delta_bias_y)

            if k%1 == 0:
                print("Epoch " + str(k) + " : Loss = " + str(self._calc_smooth_loss(loss, len(inputs))))
            
        self.save(model_file)


    def predict(self, X):
        """
        Given an input X, emi
        :param X: test input
        :return: the output class
        """
        hidden_activations, prob_values = self._forward_propagation(X)
        # return probs
        return np.argmax(prob_values)

    def save(self, model_file):
        """
        Saves the network to a file
        :param model_file: name of the file where the network should be saved
        :return: None
        """
        pickle.dump(self, open(model_file, 'wb'))

This is just a test cell, to test the working of NN. Need to remove this in the final version.

In [16]:
nn = FullyConnectedFeedForwardNN(4,8,4, activation ='relu')
inputs = []
targets = []
for i in range(1000):
    num = random.randint(0,3)
    inp = np.zeros((4,))
    inp[num] = 1
    inputs.append(inp)
    targets.append(num)

nn.train(inputs, targets, 10)
print(nn.predict([1,0,0,0]))
print(nn.predict([0,1,0,0]))
print(nn.predict([0,0,1,0]))
print(nn.predict([0,0,0,1]))

Epoch 0 : Loss = 0.5710675894286914
Epoch 1 : Loss = 0.11839317034040815
Epoch 2 : Loss = 0.03698336488697387
Epoch 3 : Loss = 0.018831883017444637
Epoch 4 : Loss = 0.011990278935063407
Epoch 5 : Loss = 0.00857658363575876
Epoch 6 : Loss = 0.00658007219402851
Epoch 7 : Loss = 0.005288426410545961
Epoch 8 : Loss = 0.00439265776468491
Epoch 9 : Loss = 0.0037391395584623907
0
1
2
3


Part 2 : Load circles dataset 

In [10]:
file = open("circles500.csv")
index = 0
circles_data = []
for line in file :
    if index == 0:
        index +=1
        continue
    x0,x1,output_class = line.split(',')
    current_row = {}
    inp = np.asarray([float(x0),float(x1)])
    current_row["inp"] = inp
    current_row["out"] = int(output_class)
    circles_data.append(current_row)

Part 2: Run Neural network model on circles dataset

In [11]:
nn=FullyConnectedFeedForwardNN(2,4,2)
inputs=[]
targets=[]
num_epocs, counter = 50, 0
training_size = int(0.9*len(circles_data))
for i in range(0,len(circles_data)):
    targets.append(circles_data[i]['out'])
    inputs.append(circles_data[i]['inp'])
nn.train(inputs[:training_size], targets[:training_size], num_epocs, model_file="Circles_NN_Model.pkl")
counter=0
for j in range(len(circles_data[training_size:])):
    s=nn.predict(circles_data[j+training_size]['inp'])
    s1=circles_data[j+training_size]['out']
    if s == s1:
        counter+=1
print("Accuracy : ",((counter*1.0)/(j+1))*100)

Epoch 0 : Loss = 0.9500350125569443
Epoch 1 : Loss = 0.6938922314870895
Epoch 2 : Loss = 0.6508353450240333
Epoch 3 : Loss = 0.6207526775829462
Epoch 4 : Loss = 0.594560670918524
Epoch 5 : Loss = 0.5708922643739173
Epoch 6 : Loss = 0.548972685082602
Epoch 7 : Loss = 0.527415603211302
Epoch 8 : Loss = 0.5076766840945053
Epoch 9 : Loss = 0.4873032196452144
Epoch 10 : Loss = 0.4655694563538947
Epoch 11 : Loss = 0.44293902021655285
Epoch 12 : Loss = 0.4183169010147366
Epoch 13 : Loss = 0.3907915786167022
Epoch 14 : Loss = 0.36242650857240544
Epoch 15 : Loss = 0.3324425545171324
Epoch 16 : Loss = 0.29633882547430196
Epoch 17 : Loss = 0.2635793183376244
Epoch 18 : Loss = 0.23744725471050457
Epoch 19 : Loss = 0.21856271813984315
Epoch 20 : Loss = 0.20289996143972094
Epoch 21 : Loss = 0.18974257717033277
Epoch 22 : Loss = 0.17824899101194153
Epoch 23 : Loss = 0.16834980276145603
Epoch 24 : Loss = 0.1596640279615901
Epoch 25 : Loss = 0.15197971810734728
Epoch 26 : Loss = 0.14509707101023897
Epo

Part 3 : Load CIFAR Dataset. 
Will load cat and deer train(all batches) and test samples

In [25]:
folder = "CIFAR_Dataset"
train_files = [folder+"/data_batch_"+str(i) for i in range(1,6) ]
test_file = folder+"/test_batch"
def NormalizeData(data):
    return (data - np.min(data)) / (np.max(data) - np.min(data))
def Convert_rgb_to_grayscale_and_normalize(image_vector):
    grayscale_vector = []
    individual_spec_length = int(len(image_vector)/3) 
    for i in range(individual_spec_length):
        red_value = image_vector[i]
        green_value = image_vector[i + individual_spec_length]
        blue_value = image_vector[i+ (2*individual_spec_length)]
        # New grayscale image = ( (0.3 * R) + (0.59 * G) + (0.11 * B) ).
        grayscale_value = ((0.3*red_value) + (0.59*green_value) + (0.11*blue_value))
        grayscale_vector.append(grayscale_value)
    return NormalizeData(np.asarray(grayscale_vector))
# This function taken from the CIFAR website
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
deer_samples, cat_samples = {}, {}
deer_samples['train'], cat_samples['train'] = [], []
cat_samples['test'], deer_samples['test'] = [], []
for file in train_files :
    train_data = unpickle(file)
    for i in range(len(train_data[b'labels'])):
        if train_data[b'labels'][i] == 3:
            cat_samples['train'].append(Convert_rgb_to_grayscale_and_normalize(train_data[b'data'][i].tolist()))
        if train_data[b'labels'][i] == 4:
            deer_samples['train'].append(Convert_rgb_to_grayscale_and_normalize(train_data[b'data'][i].tolist()))

test_data = unpickle(test_file)
for i in range(0,len(test_data[b'labels'])):
    if test_data[b'labels'][i] == 3:
        cat_samples['test'].append(Convert_rgb_to_grayscale_and_normalize(test_data[b'data'][i].tolist()))
    if test_data[b'labels'][i] == 4:
        deer_samples['test'].append(Convert_rgb_to_grayscale_and_normalize(test_data[b'data'][i].tolist()))
pickle.dump(cat_samples, open("cat_samples.pkl", "wb"))
pickle.dump(deer_samples, open("deer_samples.pkl", "wb"))

Test CIFAR dataset with neural network

In [28]:
nn=FullyConnectedFeedForwardNN(1024,16,2,activation = 'relu')
inputs, targets=[], []
num_epocs, counter = 50, 0
for i in range(len(cat_samples['train'])):
    inputs.append(cat_samples['train'][i])
    targets.append(0)
    inputs.append(deer_samples['train'][i])
    targets.append(1)
nn.train(inputs, targets, num_epocs)
for j in range(0,len(deer_samples['test'])):
    s=nn.predict(deer_samples['test'][j])
    if s == 1 :
        counter+=1
    s=nn.predict(cat_samples['test'][j])
    if s == 0 :
        counter+=1
print("Correctly identified test cases : ",counter)
print("Accuracy : ",((counter*1.0)/((j+1)*2))*100)

Epoch 0 : Loss = 0.6772088976761089
Epoch 1 : Loss = 0.6321344642120926
Epoch 2 : Loss = 0.6077941608883441
Epoch 3 : Loss = 0.5893005392664837
Epoch 4 : Loss = 0.581905976441496
Epoch 5 : Loss = 0.5766452197037141
Epoch 6 : Loss = 0.5733516715776886
Epoch 7 : Loss = 0.569251114120924
Epoch 8 : Loss = 0.5653865854838434
Epoch 9 : Loss = 0.5619617855103695
Epoch 10 : Loss = 0.5588451731588293
Epoch 11 : Loss = 0.5571108733215544
Epoch 12 : Loss = 0.5539469448230354
Epoch 13 : Loss = 0.5515048886483177
Epoch 14 : Loss = 0.5482896377124371
Epoch 15 : Loss = 0.5454031341072747
Epoch 16 : Loss = 0.5428533610980713
Epoch 17 : Loss = 0.5401453942986268
Epoch 18 : Loss = 0.538527932104156
Epoch 19 : Loss = 0.536398224686161
Epoch 20 : Loss = 0.5342814795974928
Epoch 21 : Loss = 0.5320108193116523
Epoch 22 : Loss = 0.5311591389787209
Epoch 23 : Loss = 0.528707298194548
Epoch 24 : Loss = 0.5270380393062113
Epoch 25 : Loss = 0.5255146018405144
Epoch 26 : Loss = 0.5239004773312612
Epoch 27 : Loss 

Part 4: Suruchi Implementation 
    L2 regularizing. Solves the problem of overfitting

In [31]:
class FullyConnectedNNWithRegulization(object):
    """
    Implementation of a Fully connected feed forward Neural Network. 
    This implementation implements only one hidden layer.
    """
    def __init__(self, input_size, hidden_size, output_size, learning_rate=0.01, reg_lambda=0.01, activation = 'relu'):
        """
        Initialize the network with input, output sizes, weights and biases
        :param input_dim: input dim
        :param hidden_size: number of hidden units
        :param output_dim: output dim
        :param learning_rate: learning rate alpha
        :param reg_lambda: regularization rate lambda
        :return: None
        """
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_size = hidden_size
        self.W_xh = np.random.randn(self.hidden_size, self.input_size) * np.sqrt(2/self.input_size) # Weight matrix for input to hidden
        self.W_hy = np.random.randn(self.output_size, self.hidden_size) * np.sqrt(2/self.hidden_size) # Weight matrix for hidden to output
        self.bias_h = np.zeros((self.hidden_size, 1)) # hidden bias
        self.bias_y = np.zeros((self.output_size, 1)) # output bias
        self.learning_rate = learning_rate
        self.reg_lambda = reg_lambda
        if(activation == 'sigmoid'):
            self.activation = self._sigmoid
            self.activation_derivative = self._sigmoid_derivative
        elif(activation == 'tanh'):
            self.activation = self._tanh
            self.activation_derivative = self._tanh_derivative
        else:
            self.activation = self._relu
            self.activation_derivative = self._relu_derivative
    
    def _relu(self,Z):
        return np.maximum(Z, 0)
    def _tanh(self,Z):
        return np.tanh(Z)
    def _sigmoid(self,Z):
        return 1/(1+np.exp(-Z))
    
    def _relu_derivative(self,Z):
        Z[Z<=0] = 0
        Z[Z>0] = 1
        return Z
    def _tanh_derivative(self,Z):
        return (1 - Z * Z)
    def _sigmoid_derivative(self,Z):
        return Z*(1-Z)

    def _forward_propagation(self, X):
        """
        Performs forward pass of the ANN
        :param X: input
        :return: hidden activations, softmax probabilities for the output
        """
        Z = np.dot(self.W_xh, np.reshape(X,(len(X),1))) + self.bias_h
        hidden_activations = self.activation(Z)
        y_s = np.exp(np.dot(self.W_hy, hidden_activations) + self.bias_y)
        prob_values = y_s/np.sum(y_s)
        return hidden_activations, prob_values
    
    def _regularize_weights(self, delta_W_hy, delta_W_xh, W_hy, W_xh):
        """
        Add regularization terms to the weights
        :param dWhy: weight derivative from hidden to output
        :param dWxh: weight derivative from input to hidden
        :param Why: weights from hidden to output
        :param Wxh: weights from input to hidden
        :return: dWhy, dWxh
        """
        delta_W_hy += self.reg_lambda * W_hy
        delta_W_xh += self.reg_lambda * W_xh
        return delta_W_hy, delta_W_xh

    def _update_parameter(self, delta_W_xh, delta_bias_h, delta_W_hy, delta_bias_y):
        """
        Update the weights and biases during gradient descent
        :param dWxh: weight derivative from input to hidden
        :param dbh: bias derivative from input to hidden
        :param dWhy: weight derivative from hidden to output
        :param dby: bias derivative from hidden to output
        :return: None
        """
        self.W_xh += -self.learning_rate * delta_W_xh
        self.bias_h += -self.learning_rate * delta_bias_h
        self.W_hy += -self.learning_rate * delta_W_hy
        self.bias_y += -self.learning_rate * delta_bias_y

    def _back_propagation(self, X, target_class, hidden_activations, prob_values):
        """
        Implementation of the backpropagation algorithm
        :param X: input
        :param t: target
        :param h_a: hidden activation from forward pass
        :param probs: softmax probabilities of output from forward pass
        :return: dWxh, dWhy, dbh, dby
        """
        delta_W_xh, delta_W_hy = np.zeros_like(self.W_xh), np.zeros_like(self.W_hy)
        delta_bias_h, delta_bias_y = np.zeros_like(self.bias_h), np.zeros_like(self.bias_y)
        
        delta_y = np.copy(prob_values)
        delta_y[target_class] -= 1
        delta_W_hy = np.dot(delta_y, hidden_activations.T)
        delta_bias_y += delta_y
        
        delta_h = np.dot(self.W_hy.T, delta_y)  # backprop into h
        delta_h_error = self.activation_derivative(hidden_activations) * delta_h # backprop through tanh nonlinearity
        delta_bias_h += delta_h_error
        
        delta_W_xh += np.dot(delta_h_error, np.reshape(X, (len(X), 1)).T)
        return delta_W_xh, delta_W_hy, delta_bias_h, delta_bias_y

    def _calc_smooth_loss(self, loss, len_samples, regularization=None):
        """
        Calculate the smoothened loss over the set of examples
        :param loss: loss calculated for a sample
        :param len_examples: total number of samples in training + validation set
        :param regularizer_type: type of regularizer like L1, L2, Dropout
        :return: smooth loss
        """
        if regularization == 'L2':
            # Add regulatization term to loss
            loss += self.reg_lambda/2 * (np.sum(np.square(self.W_xh)) + np.sum(np.square(self.W_hy)))
            return 1./len_samples * loss
        else:
            return 1./len_samples * loss

    def train(self, inputs, targets, sample_data, num_epochs,model_file = "NNModel.pkl", regularization=None):
        """
        Trains the network by performing forward pass followed by backpropagation
        :param inputs: list of training inputs
        :param targets: list of corresponding training targets
        :param validation_data: tuple of (X,y) where X and y are inputs and targets
        :param num_epochs: number of epochs for training the model
        :param regularizer_type: type of regularizer like L1, L2, Dropout
        :return: None
        """
        for k in range(num_epochs):
            loss = 0
            for i in range(len(inputs)):
                # Forward pass
                hidden_activations, prob_values = self._forward_propagation(inputs[i])
                loss += -np.log(prob_values[targets[i], 0])

                # Backpropagation
                delta_W_xh, delta_W_hy, delta_bias_h, delta_bias_y = self._back_propagation(inputs[i], targets[i], hidden_activations, prob_values)

                # Perform the parameter update with gradient descent
                self._update_parameter(delta_W_xh, delta_bias_h, delta_W_hy, delta_bias_y)
            
            for i in range(len(sample_data[0])):
                # Forward pass
                hidden_activations, prob_values = self._forward_propagation(sample_data[0][i])
                loss += -np.log(prob_values[sample_data[1][i], 0])

                # Backpropagation
                delta_W_xh, delta_W_hy, delta_bias_h, delta_bias_y = self._back_propagation(sample_data[0][i], sample_data[1][i], hidden_activations, prob_values)

                if regularization == 'L2':
                    delta_W_hy, delta_W_xh = self._regularize_weights(delta_W_hy, delta_W_xh, self.W_hy, self.W_xh)

                # Perform the parameter update with gradient descent
                self._update_parameter(delta_W_xh, delta_bias_h, delta_W_hy, delta_bias_y)
            
            if k%1 == 0:
                print("Epoch " + str(k) + " : Loss = " + str(self._calc_smooth_loss(loss, len(inputs), regularization)))
            
        self.save(model_file)


    def predict(self, X):
        """
        Given an input X, emi
        :param X: test input
        :return: the output class
        """
        hidden_activations, prob_values = self._forward_propagation(X)
        # return probs
        return np.argmax(prob_values)

    def save(self, model_file):
        """
        Saves the network to a file
        :param model_file: name of the file where the network should be saved
        :return: None
        """
        pickle.dump(self, open(model_file, 'wb'))

This is just here to test the above created NN. Should be removed in the final version

In [23]:
nn = FullyConnectedNNWithRegulization(4,8,4,activation = 'sigmoid')
inputs = []
targets = []
for i in range(1000):
    num = random.randint(0,3)
    inp = np.zeros((4,))
    inp[num] = 1
    inputs.append(inp)
    targets.append(num)

nn.train(inputs[:800], targets[:800], (inputs[800:], targets[800:]), 10, regularization='L2')
print(nn.predict([1,0,0,0]))
print(nn.predict([0,1,0,0]))
print(nn.predict([0,0,1,0]))
print(nn.predict([0,0,0,1]))

Epoch 0 : Loss = 1.6111589125018768
Epoch 1 : Loss = 1.1155681706546765
Epoch 2 : Loss = 0.6914498286388921
Epoch 3 : Loss = 0.41345894683015155
Epoch 4 : Loss = 0.26681855757713946
Epoch 5 : Loss = 0.19003807311009624
Epoch 6 : Loss = 0.1465713821792608
Epoch 7 : Loss = 0.11983594061085263
Epoch 8 : Loss = 0.10223125112392686
Epoch 9 : Loss = 0.09000693032523369
0
1
2
3


Add activation = 'relu' activation = 'sigmoid' and activation = 'tanh' 
Also try with different hidden layer neurons like 16,24,32
May also try different reg_lambda. Maybe 0.01, 0.05, 0.001
If have more time and need more thrill try changing epoch to 50,100,150

Please write accuracy values here for all configurations 

In [30]:
nn=FullyConnectedNNWithRegulization(1024,16,2)
inputs, targets=[], []
num_epocs, counter = 50, 0
for i in range(len(cat_samples['train'])):
    inputs.append(cat_samples['train'][i])
    targets.append(0)
    inputs.append(deer_samples['train'][i])
    targets.append(1)
training_size = int(0.8*len(targets))
validation_size = int(0.2*len(targets))
nn.train(inputs[:training_size], targets[:training_size],(inputs[training_size:],targets[training_size:]) ,num_epocs, regularization='L2')
for j in range(0,len(deer_samples['test'])):
    s=nn.predict(deer_samples['test'][j])
    if s == 1 :
        counter+=1
    s=nn.predict(cat_samples['test'][j])
    if s == 0 :
        counter+=1
print("Correctly identified test cases : ",counter)
print("Accuracy : ",((counter*1.0)/((j+1)*2))*100)

Epoch 0 : Loss = 0.8408397544910008
Epoch 1 : Loss = 0.7771260372899153
Epoch 2 : Loss = 0.7496690453700748
Epoch 3 : Loss = 0.7421302226733082
Epoch 4 : Loss = 0.7369559086058194
Epoch 5 : Loss = 0.733992733107598
Epoch 6 : Loss = 0.7316099818287766
Epoch 7 : Loss = 0.7294390241215375
Epoch 8 : Loss = 0.7279853091363935
Epoch 9 : Loss = 0.7258437563352804
Epoch 10 : Loss = 0.7225448639840865
Epoch 11 : Loss = 0.7200122639905867
Epoch 12 : Loss = 0.7176054904354143
Epoch 13 : Loss = 0.7167209574924347
Epoch 14 : Loss = 0.7147666112825061
Epoch 15 : Loss = 0.7153740968628881
Epoch 16 : Loss = 0.7143383553214933
Epoch 17 : Loss = 0.7120940183104457
Epoch 18 : Loss = 0.7118555344473881
Epoch 19 : Loss = 0.7112121873028133
Epoch 20 : Loss = 0.7110898356033494
Epoch 21 : Loss = 0.7105156880008586
Epoch 22 : Loss = 0.7102801298089656
Epoch 23 : Loss = 0.7095130306009473
Epoch 24 : Loss = 0.7085770789539537
Epoch 25 : Loss = 0.707661436307962
Epoch 26 : Loss = 0.7070107007150116
Epoch 27 : Lo